In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# For reproducibility
torch.manual_seed(1)

- Discrete Probability Distribution (이산적인 확률 분포)

- Softmax
  - convert numbers to probabilities with softmax.
$$P(class = i) = \frac{e^i}{\sum{e^i}}$$

- 철수가 가위를 냈을때, 그 다음에 주먹을 낼 확률
$$ P(주먹|가위) = ? $$
$$ P(가위|가위) = ? $$
$$ P(보|가위) = ? $$

In [5]:
z = torch.FloatTensor([1, 2, 3])

# max result = max=([0, 0, 1])

PyTorch has a softmax fuction.
- max를 뽑을 때 soft 하게 max를 뽑아줌
- max값의 확률 값을 리턴해줌

In [6]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


- [0.0900, 0.2447, 0.6652] 일때, 

  - hypothesis[0] = 0.09 인 확률 = $ \frac {e^1}{e^1 + e^2 + e^3} $
  - hypothesis[1] = 0.24 인 확률 = $ \frac {e^2} {e^1 + e^2 + e^3} $

Since they are probabilities, they should add up to 1. Let's do a sanity check.

In [7]:
hypothesis.sum()

tensor(1.)

- Cross Entropy Loss (Low-level)
$$ H(P, Q) = -\mathbb{E}{_x\sim_P(x)}[logQ(x)]= - \sum_{x\in X}P(x)logQ(x) $$
  - 두개의 확률 분포가 주어졌을때, 얼마나 두개의 분포가 비슷한지를 나타내는 수치
  - 철수가 가위를 냈을때, 다음에 무엇을 낼지 확률 분포가 있을때,
    cross entropy를 구해서 최소화를 하게 되면 Q2 -> Q1 -> P에 가까워질수 있다
  
For multi-class classification, we use the cross entropy loss.
$$ L = \frac{1}{N} \sum-y log(\hat y)$$
- y = P(x)
- $\hat y$ = $P_\Theta(X)$


- where y_hat is the predicted probability and y is the correct probability (0 or 1).

In [13]:
z = torch.rand(3, 5, requires_grad=True) # |Z| = (3, 5)
print(z)

# do softmax row 1th. this is a prediction value.
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2616, 0.7713, 0.3785, 0.9980, 0.9008],
        [0.4766, 0.1663, 0.8045, 0.6552, 0.1768],
        [0.8248, 0.8036, 0.9434, 0.2197, 0.4177]], requires_grad=True)
tensor([[0.1287, 0.2142, 0.1446, 0.2687, 0.2438],
        [0.1977, 0.1450, 0.2744, 0.2364, 0.1465],
        [0.2317, 0.2268, 0.2609, 0.1265, 0.1542]], grad_fn=<SoftmaxBackward>)


In [10]:
# classes = 5
# samples = 3

y = torch.randint(5, (3,)).long()
print(y)

tensor([2, 1, 0])


In [14]:
y_one_hot = torch.zeros_like(hypothesis) # |y_one_hot| = (3, 5)

# demension=1, |y|=(3, ) -> unsqueeze -> (3, 1)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # _ 포함된 함수는 inplace 함수임 (객체에 바로 반영)

#              [2]
# [2, 1, 0] => [1]
#              [0]

tensor([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [16]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.7758, grad_fn=<MeanBackward0>)


In [17]:
# Cross-entropy Loss with torch.nn.functional
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-2.0505, -1.5408, -1.9337, -1.3141, -1.4114],
        [-1.6209, -1.9313, -1.2930, -1.4423, -1.9207],
        [-1.4625, -1.4837, -1.3438, -2.0675, -1.8696]], grad_fn=<LogBackward>)

In [18]:
# High level
F.log_softmax(z, dim=1)

tensor([[-2.0505, -1.5408, -1.9337, -1.3141, -1.4114],
        [-1.6209, -1.9313, -1.2930, -1.4423, -1.9207],
        [-1.4625, -1.4837, -1.3438, -2.0675, -1.8696]],
       grad_fn=<LogSoftmaxBackward>)

In [19]:
# low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.7758, grad_fn=<MeanBackward0>)

In [20]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y) # Negative Log Likelihood

tensor(1.7758, grad_fn=<NllLossBackward>)

In [22]:
# PyTorch also has F.cross_entropy that combines F.log_softmax() and F.nll_loss().
# Cross-entropy Loss with torch.nn.functional

F.cross_entropy(z, y)

tensor(1.7758, grad_fn=<NllLossBackward>)

Training with Low-level Cross Entropy Loss

In [24]:
# |x_train| = (m, 4)
x_train = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5],
           [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]

# |y_train| = (m,)
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

Training with Low-level Cross Entropy Loss

In [27]:
# model init
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# set optimizer
optimizer = optim.SGD((W, b), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # Cost 로 H(x) 계선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번 마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


Training with F.cross_entropy

In [28]:
# model init
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# set optimizer
optimizer = optim.SGD((W, b), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train)
    
    # cost 로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


High-level Implementation with nn.Module

In [33]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # output 이 3 : 4 -> 3
    
    def forward(self, x):
        return self.linear(x) # |X| = (m, 4) => (m, 3)

In [34]:
model = SoftmaxClassifierModel()

In [36]:
#High-level Implementation with nn.Module
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # |x_train| = (m, 4)
    # |prediction| = (m, 3)
    
    # cost 계산
    cost = F.cross_entropy(prediction, y_train) # |y_train| = (m,)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 0.249607
Epoch  100/1000 Cost: 0.234399
Epoch  200/1000 Cost: 0.222989
Epoch  300/1000 Cost: 0.212605
Epoch  400/1000 Cost: 0.203111
Epoch  500/1000 Cost: 0.194398
Epoch  600/1000 Cost: 0.186374
Epoch  700/1000 Cost: 0.178961
Epoch  800/1000 Cost: 0.172094
Epoch  900/1000 Cost: 0.165714
Epoch 1000/1000 Cost: 0.159774
